# Goal: See how much of an effect our wasting treatment and prevention are having

Model version: `v4.0_wasting_treatment`

It looks like we're averting around 1% of total DALYs, but we can't yet check the reduction in wasting prevalence because of a bug in the observers.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model4
ndbs
Wed Sep 22 13:47:18 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [47]:
csr.get_count_data_path(4.0)

'/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.0_wasting_treatment/ciff_sam/2021_09_20_14_45_25/count_data/'

# Load data and compute total person time

Note that this model run was only partially complete.

In [3]:
data = csr.VivariumResults.cleaned_from_model_spec(4.0, '2021_09_20_14_45_25')
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count']

In [16]:
data.compute_total_person_time()
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'cause_state_person_time',
 'cause_transition_count',
 'person_time']

# Look at total DALYs in each scenario and calculate averted DALYs

Start with YLLs

In [4]:
data.ylls

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,1955.068412,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,266.589544,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,177.735076,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
51836,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,susceptible_to_child_wasting,2_to_4
51837,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,mild_child_wasting,2_to_4
51838,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,0.000000,moderate_acute_malnutrition,2_to_4
51839,male,2026,severe_acute_malnutrition,ylls,946,wasting_treatment,600.389858,severe_acute_malnutrition,2_to_4


In [7]:
averted_ylls = vop.averted(data.ylls, 'baseline')
averted_ylls

,sex,year,cause,measure,input_draw,wasting_state,age,scenario,subtracted_from,value
0,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,1-5_months,sqlns,baseline,0.000000e+00
1,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,1-5_months,wasting_treatment,baseline,9.094947e-13
2,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,12_to_23_months,sqlns,baseline,0.000000e+00
3,female,2022,diarrheal_diseases,ylls,29,mild_child_wasting,12_to_23_months,wasting_treatment,baseline,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,early_neonatal,sqlns,baseline,0.000000e+00
34557,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,early_neonatal,wasting_treatment,baseline,0.000000e+00
34558,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,late_neonatal,sqlns,baseline,0.000000e+00
34559,male,2026,severe_acute_malnutrition,ylls,946,susceptible_to_child_wasting,late_neonatal,wasting_treatment,baseline,0.000000e+00


# Look at averted YLLs and total YLLs in each scenario

Averted YLLs more than double when implementing prevention (SQLNS) in addition to wasting treatment.

We are only averting around 1% of total DALYs in each scenario.

In [42]:
# fraction of total DALYs averted in wasting_treatment and sqlns scenarios
np.array([20735.562130, 43164.853815]) / (3.381217e+06)

array([0.00613257, 0.01276607])

In [9]:
# Stratifying by nothing tallies results over all strata for each scenario and draw
# Calling vop.describe() summarizes the distribution over draws
vop.describe(vop.stratify(averted_ylls, []))

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
sqlns,12.0,43164.853815,12319.644041,23539.840962,23799.624129,43785.437467,61545.600369,62313.180855
wasting_treatment,12.0,20735.562130,5208.949570,12724.929870,12891.506248,22189.594518,26967.460296,27483.009862


In [10]:
vop.describe(vop.stratify(data.ylls, []))

,count,mean,std,min,2.5%,50%,97.5%,max
scenario,,,,,,,,
baseline,12.0,3.381217e+06,470156.842603,2.389399e+06,2.546420e+06,3.378761e+06,3.965572e+06,3.979998e+06
sqlns,12.0,3.338052e+06,460974.131115,2.365859e+06,2.522620e+06,3.340226e+06,3.910364e+06,3.920476e+06
wasting_treatment,12.0,3.360482e+06,467072.353095,2.376674e+06,2.533528e+06,3.362078e+06,3.943280e+06,3.957580e+06


# Stratify overall YLLs and YLDs by wasting state

Most DALYs are in the sucecptible and mild states, whereas our treatments affect the moderate and severe states. As a fraction of MAM and SAM DALYS, the averted DALYs are more like 3% or 7% (but note that the numerators are not stratified by wasting state).

In [41]:
# fraction of MAM and SAM DALYs averted in wasting_treatment and sqlns scenarios
np.array([20735.562130, 43164.853815]) / (416103.353244+179741.929651)

array([0.03480025, 0.07244306])

In [12]:
vop.describe(vop.stratify(data.ylls, ['wasting_state']))['mean'].unstack()

wasting_state,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
scenario,,,,
baseline,701967.816486,416103.353244,179741.929651,2.083404e+06
sqlns,719418.743836,372708.240153,161775.279802,2.084150e+06
wasting_treatment,714090.334637,391068.591138,168158.760607,2.087164e+06


In [13]:
vop.describe(vop.stratify(data.ylds, ['wasting_state']))['mean'].unstack()

wasting_state,mild_child_wasting,moderate_acute_malnutrition,severe_acute_malnutrition,susceptible_to_child_wasting
scenario,,,,
baseline,3178.745353,2551.106175,10591.101370,6012.638442
sqlns,3294.938949,2044.580741,8688.154923,5953.380232
wasting_treatment,3297.773848,2255.831745,9330.097950,6041.838814


# Compute fraction of total population in each wasting state in each scenario

We want to see how much our interventions decrease the prevalence of wasting.

In [14]:
data.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,144.720055,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,149.426420,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,covered,2_to_4
34557,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,uncovered,2_to_4
34558,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,uncovered,covered,2_to_4
34559,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,15488.375086,uncovered,uncovered,2_to_4


## Get person time in each wasting state and total person time, filter out pre-intervention years

In [37]:
pt_by_wasting_state = vop.stratify(
    data.wasting_state_person_time.query("year > '2022'"), ['wasting_state', 'measure'])
pt_by_wasting_state

,wasting_state,measure,input_draw,scenario,value
0,mild_child_wasting,state_person_time,29,baseline,6.445073e+05
1,mild_child_wasting,state_person_time,29,sqlns,6.492546e+04
2,mild_child_wasting,state_person_time,29,wasting_treatment,6.181197e+04
3,mild_child_wasting,state_person_time,223,baseline,5.909678e+05
...,...,...,...,...,...
140,susceptible_to_child_wasting,state_person_time,829,wasting_treatment,2.003793e+05
141,susceptible_to_child_wasting,state_person_time,946,baseline,2.086413e+06
142,susceptible_to_child_wasting,state_person_time,946,sqlns,2.078488e+05
143,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,2.070319e+05


In [30]:
data.person_time

,year,sex,age,input_draw,scenario,value,measure
0,2022,female,1-5_months,29,baseline,31642.357290,person_time
1,2022,female,1-5_months,29,sqlns,31642.357290,person_time
2,2022,female,1-5_months,29,wasting_treatment,31642.357290,person_time
3,2022,female,1-5_months,223,baseline,28922.950034,person_time
...,...,...,...,...,...,...,...
2516,2023,male,all_ages,946,wasting_treatment,38958.186174,person_time
2517,2024,male,all_ages,946,wasting_treatment,39283.652293,person_time
2518,2025,male,all_ages,946,wasting_treatment,39258.629706,person_time
2519,2026,male,all_ages,946,wasting_treatment,39101.646817,person_time


In [38]:
pt = vop.stratify(data.person_time.query("age != 'all_ages' and year > '2022'"), ['measure'])
pt

,measure,input_draw,scenario,value
0,person_time,29,baseline,3.004774e+06
1,person_time,29,sqlns,3.011619e+05
2,person_time,29,wasting_treatment,3.010839e+05
3,person_time,223,baseline,2.745973e+06
...,...,...,...,...
32,person_time,829,wasting_treatment,2.949611e+05
33,person_time,946,baseline,3.044920e+06
34,person_time,946,sqlns,3.047343e+05
35,person_time,946,wasting_treatment,3.046905e+05


## Calculate prevalence of each wasting state

In [39]:
pop_fraction_by_wasting_state = vop.ratio(
    pt_by_wasting_state,
    pt,
    strata = [],
    numerator_broadcast='wasting_state'
)
pop_fraction_by_wasting_state

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,0.214494,state_person_time,person_time,1
1,29,baseline,moderate_acute_malnutrition,0.079999,state_person_time,person_time,1
2,29,baseline,severe_acute_malnutrition,0.019481,state_person_time,person_time,1
3,29,baseline,susceptible_to_child_wasting,0.686026,state_person_time,person_time,1
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.203567,state_person_time,person_time,1
141,946,wasting_treatment,moderate_acute_malnutrition,0.090914,state_person_time,person_time,1
142,946,wasting_treatment,severe_acute_malnutrition,0.026037,state_person_time,person_time,1
143,946,wasting_treatment,susceptible_to_child_wasting,0.679483,state_person_time,person_time,1


## It looks like SAM and MAM prevalence are going _up_ in our alternative scenarios!

This is likely due to a bug in the observers. Rajan says in a [Slack message from Wed 9/22, 2021](https://ihme.slack.com/archives/C018BLX2JKT/p1632347346165200):

> **Rajan Mudambi**  2:49 PM
>
> @Nathaniel Blair-Stahn Just as a heads up, in the partial runs we got yesterday the alternative wasting scenarios doesn't capture any wasting person time or transitions in the covered population due to a bug I found today. The baseline scenario should be fine as well as the other count data. I have a new run underway with the fix.

In [40]:
vop.describe(pop_fraction_by_wasting_state)['mean'].unstack()

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         1          state_person_time baseline                     0.215127   
                                                 sqlns                        0.214620   
                                                 wasting_treatment            0.206757   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                              0.079530   
                                                 sqlns                                 0.081500   
                                                 wasting_treatment                     0.090743   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         1          state_person_time baseline                            0.018881   
                                                 sqlns                               0.020187   
                                                 wasting_treatment                   0.022081   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                               0.686462  
                                                 sqlns                                  0.683693  
                                                 wasting_treatment                      0.680418

# Check that wasting prevalence is same in all scenarios before interventions start

Yes, looks good.

In [43]:
pt_by_wasting_state_pre_2023 = vop.stratify(
    data.wasting_state_person_time.query("year <= '2022'"), ['wasting_state', 'measure'])
pt_by_wasting_state_pre_2023

,wasting_state,measure,input_draw,scenario,value
0,mild_child_wasting,state_person_time,29,baseline,155890.358658
1,mild_child_wasting,state_person_time,29,sqlns,155890.358658
2,mild_child_wasting,state_person_time,29,wasting_treatment,155890.358658
3,mild_child_wasting,state_person_time,223,baseline,142218.902122
...,...,...,...,...,...
140,susceptible_to_child_wasting,state_person_time,829,wasting_treatment,495222.392882
141,susceptible_to_child_wasting,state_person_time,946,baseline,514961.056810
142,susceptible_to_child_wasting,state_person_time,946,sqlns,514961.056810
143,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,514961.056810


In [44]:
pt_pre_2023 = vop.stratify(data.person_time.query("age != 'all_ages' and year <= '2022'"), ['measure'])
pt_pre_2023

,measure,input_draw,scenario,value
0,person_time,29,baseline,737352.550308
1,person_time,29,sqlns,737352.550308
2,person_time,29,wasting_treatment,737352.550308
3,person_time,223,baseline,674165.841205
...,...,...,...,...
32,person_time,829,wasting_treatment,716711.891855
33,person_time,946,baseline,746961.399042
34,person_time,946,sqlns,746961.399042
35,person_time,946,wasting_treatment,746961.399042


In [45]:
pop_fraction_by_wasting_state_pre_2023 = vop.ratio(
    pt_by_wasting_state_pre_2023,
    pt_pre_2023,
    strata = [],
    numerator_broadcast='wasting_state'
)
pop_fraction_by_wasting_state_pre_2023

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier
0,29,baseline,mild_child_wasting,0.211419,state_person_time,person_time,1
1,29,baseline,moderate_acute_malnutrition,0.078835,state_person_time,person_time,1
2,29,baseline,severe_acute_malnutrition,0.019252,state_person_time,person_time,1
3,29,baseline,susceptible_to_child_wasting,0.690494,state_person_time,person_time,1
...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,0.209558,state_person_time,person_time,1
141,946,wasting_treatment,moderate_acute_malnutrition,0.079847,state_person_time,person_time,1
142,946,wasting_treatment,severe_acute_malnutrition,0.021187,state_person_time,person_time,1
143,946,wasting_treatment,susceptible_to_child_wasting,0.689408,state_person_time,person_time,1


In [46]:
vop.describe(pop_fraction_by_wasting_state_pre_2023)['mean'].unstack()

wasting_state                                                       mild_child_wasting  \
denominator_measure multiplier numerator_measure scenario                                
person_time         1          state_person_time baseline                     0.211599   
                                                 sqlns                        0.211599   
                                                 wasting_treatment            0.211599   

wasting_state                                                       moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                                0.0784   
                                                 sqlns                                   0.0784   
                                                 wasting_treatment                       0.0784   

wasting_state                                                       severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure scenario                                       
person_time         1          state_person_time baseline                            0.018785   
                                                 sqlns                               0.018785   
                                                 wasting_treatment                   0.018785   

wasting_state                                                       susceptible_to_child_wasting  
denominator_measure multiplier numerator_measure scenario                                         
person_time         1          state_person_time baseline                               0.691215  
                                                 sqlns                                  0.691215  
                                                 wasting_treatment                      0.691215